In [ ]:
# ==========================================================
#                       LIBRARIES
# ==========================================================

import pandas as pd
import numpy as np

from plotly import graph_objects as go
from plotly.subplots import make_subplots


In [51]:
# load data

gc = pd.read_csv('x_y_z_bin_gc.csv').iloc[:, 1:]
com = pd.DataFrame(np.load('x_y_z_dist_com_dist_rm.npy'), columns=['x', 'y', 'z', 'dist_com', 'dist_rm'])
sasa = pd.DataFrame(np.load('x_y_z_dist_surf.npy'), columns=['x', 'y', 'z', 'dist_surf'])
surface_coords_scaled = pd.DataFrame(np.load('surface_coords_scaled.npy'), columns=['x', 'y', 'z'])


In [52]:
data_combined = pd.concat([gc, com[['dist_com', 'dist_rm']], sasa['dist_surf']], axis=1)

data_combined


,x,y,z,bin,gc_content_percentage,dist_com,dist_rm,dist_surf
0,-0.3264,0.3988,-0.2650,0,33.65,0.581433,0.238979,0.097393
1,-0.3026,0.4367,-0.2681,500000,43.05,0.596533,0.200534,0.140895
2,-0.2171,0.5026,-0.2616,1000000,60.50,0.606912,0.119064,0.177681
3,-0.0999,0.6025,-0.2160,1500000,54.07,0.645951,0.121146,0.199922
4,0.0170,0.6524,-0.1841,2000000,58.55,0.674883,0.223344,0.188560
...,...,...,...,...,...,...,...,...
446,-0.3537,0.2988,0.4387,240500000,39.82,0.635022,0.139890,0.259115
447,-0.3457,0.2375,0.4932,241000000,39.81,0.644606,0.141894,0.321318
448,-0.3446,0.1994,0.5161,241500000,40.06,0.649107,0.150746,0.327303
449,-0.3101,0.2030,0.5102,242000000,40.85,0.627557,0.145351,0.351124


In [53]:
def standard_scaler(col):
    return (col - col.mean()) / col.std()

# Columns to scale
cols_to_scale = ['gc_content_percentage', 'dist_com', 'dist_rm', 'dist_surf']

# Apply scaler only to selected columns
data_combined[cols_to_scale] = data_combined[cols_to_scale].apply(standard_scaler)

data_combined


,x,y,z,bin,gc_content_percentage,dist_com,dist_rm,dist_surf
0,-0.3264,0.3988,-0.2650,0,-1.625260,-0.780695,0.949264,-2.399716
1,-0.3026,0.4367,-0.2681,500000,0.306620,-0.676277,0.429941,-1.433268
2,-0.2171,0.5026,-0.2616,1000000,3.892928,-0.604512,-0.670555,-0.616046
3,-0.0999,0.6025,-0.2160,1500000,2.571440,-0.334563,-0.642437,-0.121955
4,0.0170,0.6524,-0.1841,2000000,3.492166,-0.134502,0.738059,-0.374361
...,...,...,...,...,...,...,...,...
446,-0.3537,0.2988,0.4387,240500000,-0.357207,-0.410132,-0.389234,1.193080
447,-0.3457,0.2375,0.4932,241000000,-0.359262,-0.343859,-0.362173,2.574957
448,-0.3446,0.1994,0.5161,241500000,-0.307882,-0.312736,-0.242599,2.707926
449,-0.3101,0.2030,0.5102,242000000,-0.145522,-0.461751,-0.315475,3.237118


In [54]:
line_dict = dict(
    width=5
)

data = go.Scatter3d(
    x=data_combined['x'],
    y=data_combined['y'],
    z=data_combined['z'],
    mode='lines',
    line=line_dict
)


fig = go.Figure(data)
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
    ),
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    paper_bgcolor='rgba(0,0,0,0)',
    showlegend=False
)

fig.show()


## Iteration 1

In [55]:
# Your DataFrame with x, y, z and feature columns
feature_cols = ['gc_content_percentage', 'dist_com', 'dist_rm', 'dist_surf']
fig = go.Figure()

# Initial trace
initial_col = feature_cols[0]

fig.add_trace(go.Scatter3d(
    x=data_combined['x'],
    y=data_combined['y'],
    z=data_combined['z'],
    mode='lines',
    line=dict(
        width=9,
        color=data_combined[initial_col],
        colorscale='Turbo',
        cmin=data_combined[feature_cols].min().min(),  # Global min across all features
        cmax=data_combined[feature_cols].max().max(),  # Global max across all features
        showscale=True,
        colorbar=dict(title=initial_col)
    )
))

# Create buttons for dropdown
buttons = []
for col in feature_cols:
    buttons.append(
        dict(
            label=col.replace('_', ' ').title(),  # Nicer labels
            method='restyle',
            args=[
                {
                    'line.color': [data_combined[col]],
                    'line.colorbar.title': col
                }
            ]
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.02,
            xanchor='left',
            y=1.02,
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='gray',
            borderwidth=1
        )
    ],
    title='3D Chromosome Plot - Select Color Feature',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
    ),
    width=1000,
    height=800,
    margin=dict(l=0, r=0, b=0, t=0, pad=0)
)

fig.show()

## Iteration 2

In [66]:
# Calculate bounds from both line and points data
all_x = np.concatenate([data_combined['x'], surface_coords_scaled['x']])
all_y = np.concatenate([data_combined['y'], surface_coords_scaled['y']])
all_z = np.concatenate([data_combined['z'], surface_coords_scaled['z']])

x_range = [all_x.min(), all_x.max()]
y_range = [all_y.min(), all_y.max()]
z_range = [all_z.min(), all_z.max()]

# Your DataFrame with x, y, z and feature columns
feature_cols = ['gc_content_percentage', 'dist_com', 'dist_rm', 'dist_surf']
fig = go.Figure()

# Initial trace (line plot)
initial_col = feature_cols[0]

fig.add_trace(go.Scatter3d(
    x=data_combined['x'],
    y=data_combined['y'],
    z=data_combined['z'],
    mode='lines',
    line=dict(
        width=9,
        color=data_combined[initial_col],
        colorscale='Turbo',
        cmin=data_combined[feature_cols].min().min(),
        cmax=data_combined[feature_cols].max().max(),
        showscale=True,  # Show colorbar
        colorbar=dict(title=''),  # Remove colorbar title
    ),
    name='Chromosome Line'
))

# Add 3D points trace
fig.add_trace(go.Scatter3d(
    x=surface_coords_scaled['x'],
    y=surface_coords_scaled['y'],
    z=surface_coords_scaled['z'],
    mode='markers',
    marker=dict(size=2, color='skyblue', opacity=0.1),
    name='Surface Points'
))

# Create buttons for dropdown (color selection) - Use 'update' instead of 'restyle'
buttons = []
for col in feature_cols:
    buttons.append(
        dict(
            label=col.replace('_', ' ').title(),
            method='update',
            args=[
                {
                    'line.color': [data_combined[col]],
                    'line.colorbar.title': ''  # Keep title empty when updating
                },
                {}  # Empty layout update to preserve camera
            ]
        )
    )

# Add button to toggle points visibility
toggle_points_button = dict(
    label='Toggle Surface Points',
    method='update',
    args=[
        {'visible': [True, True]},
        {}  # Empty layout to preserve camera
    ],
    args2=[
        {'visible': [True, False]},
        {}  # Empty layout to preserve camera
    ]
)

fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.02,
            xanchor='left',
            y=1.02,
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='gray',
            borderwidth=1
        ),
        dict(
            type='buttons',
            buttons=[toggle_points_button],
            direction='right',
            x=0.02,
            xanchor='left',
            y=0.95,
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='gray',
            borderwidth=1
        )
    ],
    title='',
    uirevision='constant',
    showlegend=True,
    legend=dict(
        title_text='',
        visible=True
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent plot background
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper background
    scene=dict(
        xaxis=dict(
            visible=False,
            range=x_range,
            autorange=False
        ),
        yaxis=dict(
            visible=False,
            range=y_range,
            autorange=False
        ),
        zaxis=dict(
            visible=False,
            range=z_range,
            autorange=False
        ),
        aspectmode='data',
        bgcolor='rgba(0,0,0,0)',  # Transparent scene background
    ),
    width=1000,
    height=800,
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
)

fig.show()

## Iteration 3